**MIMO PROTOYPE**:

Here we try our hand at making MIMO work, and testing its validity with a synthetic toy dataset. The notebook contains the following: 
- Create toy dataset and dataloaders - toy regression data & MNIST?
- Simple MIMO model - regression and classification
- Training and results on toy dataset

For the classification task, we make the following plots:
- Reliability: confidence vs error - for classification
- Expected calibration errors - for classification
- Diveristity plots - for classification and regression

For the regression task, we plot the mean prediction and a 95% confidence interval.

In [ ]:
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
# from sklearn.manifold import TSNE


def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(1871)

# Regression

***Making the dataset***

We start by making a toy dataset, which will enable us to better understand the network. Since we have full control over the data, we can see how changes impact MIMO.

The most basic problems we can solve with a MIMO-configured neural network is regression. Classification would also be a somewhat simple problem to solve. 

We first generate training data from the function
$$y = x + 0.3 \sin{(2\pi (x+ \epsilon)) + 0.3 \sin{(4\pi (x+\epsilon))}} $$
as described by Blundell et al.

We use $N_{train}=500$ training points and $N_{test}$ testing points. 

In [ ]:
# Regression data function
f = lambda x, epsilon: x + 0.3 * np.sin(2*np.pi * (x+epsilon)) + 0.3 * np.sin(4 * np.pi * (x+epsilon)) + epsilon

def generate_data(N, lower, upper, std, f=f):
    # create data
    x = np.linspace(lower, upper, N)

    y = []
    for i in range(N):
        epsilon = np.random.normal(0, std)
        y.append(f(x[i], epsilon))
    return x, y

# Generate train data
N_train = 2000
x, y = generate_data(N_train, lower=-0.25, upper=1, std=0.02)

# Generate validation data
N_val = 500
x_val, y_val = generate_data(N_val, lower=-0.25, upper=1, std=0.02)

# Generate test data
N_test = 500
x_test, y_test = generate_data(N_test, lower=-0.5, upper=1.5, std=0.02)

line = f(x_test, 0)

We make a custom toy dataset class for the MIMO model. During training, the $M$ subnetworks in the model each get a different batch of training samples. We can do this by multiplying the batch size with a factor $M$ such that the effective batch size is:
$$
batch\_size^* = \frac{batch\_size}{M}
$$
This is implemented using a custom collate function.

During inference, the $M$ subnetworks in the model each get the same input. We implement this by multiplying the input $M$ times using another collate function.

In [ ]:
class ToyDataset(Dataset):
    """Custom toy dataset"""

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):

        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)
    
def train_collate_fn(batch, M):
    """Collate function for training MIMO"""
    
    x, y = zip(*batch)
    
    x_chunks = torch.stack(torch.chunk(torch.tensor(x), M, dim=0), dim=1)
    y_chunks = torch.stack(torch.chunk(torch.tensor(y), M, dim=0), dim=1)

    return x_chunks, y_chunks

def test_collate_fn(batch, M):
    """Collate function for testing MIMO"""
    
    x, y = zip(*batch)
    x = torch.tensor(x)[:,None].repeat(1,M)
    y = torch.tensor(y)[:,None].repeat(1,M)

    return x, y

def naive_collate_fn(batch, M):
    """Collate function for naive multiheaded model"""

    x, y = zip(*batch)
    x = torch.tensor(x)[:,None]
    y = torch.tensor(y)[:,None].repeat(1,M)

    return x, y

In [ ]:
# seed workers for reproducibility
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

M = 2
traindata = ToyDataset(x, y)
trainloader = DataLoader(traindata, batch_size=60*M, shuffle=True, collate_fn=lambda x: train_collate_fn(x, M), drop_last=True, worker_init_fn=seed_worker, generator=g)
naivetrainloader = DataLoader(traindata, batch_size=60, shuffle=True, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False, worker_init_fn=seed_worker, generator=g)

valdata = ToyDataset(x_val, y_val)
valloader = DataLoader(valdata, batch_size=60, shuffle=False, collate_fn=lambda x: test_collate_fn(x, M), drop_last=False)
naivevalloader = DataLoader(valdata, batch_size=60, shuffle=False, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False)

testdata = ToyDataset(x_test, y_test)
testloader = DataLoader(testdata, batch_size=N_test, shuffle=False, collate_fn=lambda x: test_collate_fn(x, M), drop_last=False)
naivetestloader = DataLoader(testdata, batch_size=N_test, shuffle=False, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False)



# Dataloader properties
Testing shape of output from train and test loaders

The ``trainloader`` should output a batch where columns are different.

In [ ]:
for x_train,y_train in trainloader:
    print(x_train.shape)
    print(y_train.shape)
    break

print(x_train)

The MIMO dataloaders should output a batch with size $(batch\_size, M)$

In [ ]:
for test_x, test_y in testloader:
    print(test_x.shape)
    print(test_y.shape)
    break


The dataloader for the naive multiheaded model should output a batch with size $(batch\_size, 1)$ for the features $x$ and labels (batch_size, M) for $y$.

In [ ]:
for naive_x_train, naive_y_train in naivetrainloader:
    print(naive_x_train.shape)
    print(naive_y_train.shape)
    break


In [ ]:
# plot data
fig, ax = plt.subplots(1,1, figsize=(12,12))

ax.plot(x, y, '.', label='Train data', color='orange')
ax.plot(x_test, y_test, '.', label='Test data', color='blue')
ax.plot(x_test, line, '--', label='true function', color='red')
ax.legend()
ax.set_title("One-dimensional dataset")
plt.show()

***Building the model***

The model is a MIMO configured neural network. To keep it simple for now we choose to have $M=2$, so we use 2 subnetworks.
The neural network will consist of two feed-forward aka linear layers with 32 and 128 hidden units in each. For activation function we will use ReLU as it is the norm within the field.

We choose the number of hidden units we did, so that our architecture would be the same as the one given in the MIMO paper when they fit a MIMO model to this synthetic data.

We make sure that we can get both the averaged output (ensemble output) and individual ensemble member output. This will help us illustrate the diversity of the ensemble members.

In [ ]:
# Define the model
hidden_units = 32
hidden_units2 = 128

def init_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

class MIMONetwork(nn.Module):
    def __init__(self, n_subnetworks):
        super().__init__()
        self.n_subnetworks = n_subnetworks
        self.model = torch.nn.Sequential(
            nn.Linear(self.n_subnetworks, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units,hidden_units2),
            nn.ReLU(),
            nn.Linear(hidden_units2, self.n_subnetworks)
        )


    def forward(self, x):
        individual_outputs = self.model(x)
        output = torch.mean(individual_outputs, dim=1)
        return output, individual_outputs

MIMO_model = MIMONetwork(n_subnetworks=M)
MIMO_model.apply(init_weights)

class NaiveNetwork(nn.Module):
    def __init__(self, n_subnetworks):
        super().__init__()
        self.n_subnetworks = n_subnetworks
        self.model = torch.nn.Sequential(
            nn.Linear(1, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units,hidden_units2),
            nn.ReLU(),
            nn.Linear(hidden_units2, self.n_subnetworks)
        )

    def forward(self, x):
        individual_outputs = self.model(x)
        output = torch.mean(individual_outputs, dim=1)
        return output, individual_outputs

naive_model = NaiveNetwork(n_subnetworks=M)
naive_model.apply(init_weights)

In [ ]:
# weight distribution

weights_mimo = []
for param in MIMO_model.parameters():
    weights_mimo.extend(param.flatten().detach().numpy())

weights_naive = []
for param in naive_model.parameters():
    weights_naive.extend(param.flatten().detach().numpy())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.hist(weights_mimo, bins=50, alpha=0.5, label='MIMO_model')
ax1.set_xlabel('Weight Value')
ax1.set_ylabel('Frequency')
ax1.set_title('Weight Distribution - MIMO Model')

ax2.hist(weights_naive, bins=50, alpha=0.5, label='naive_model')
ax2.set_xlabel('Weight Value')
ax2.set_ylabel('Frequency')
ax2.set_title('Weight Distribution - Naive Model')

plt.tight_layout()
plt.show()

In [ ]:
# useful functions 🤖

def train(model, optimizer, trainloader, valloader, epochs=500, model_name='MIMO', val_every_n_epochs=10):

    losses = []
    val_losses = []

    best_val_loss = np.inf

    for e in tqdm(range(epochs)):
        
        for x_, y_ in trainloader:

            model.train()

            x_,y_ = x_.float(), y_.float()

            optimizer.zero_grad()

            output, individual_outputs = model(x_)
            loss = nn.functional.mse_loss(individual_outputs, y_)

            loss.backward()
            optimizer.step()

            losses.append(loss.item())  

        if (e+1) % val_every_n_epochs == 0:
            model.eval()

            val_loss_list = []
            with torch.no_grad():
                for val_x, val_y in valloader:
                    val_x, val_y = val_x.float(), val_y.float()
                    val_output, val_individual_outputs = model(val_x)
                    val_loss = nn.functional.mse_loss(val_individual_outputs, val_y)
                    val_loss_list.append(val_loss.item())

            val_losses.extend(val_loss_list)
            mean_val_loss = np.mean(val_loss_list)
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                torch.save(model, f'{model_name}.pt')
            # print(f"Mean validation loss at epoch {e}: {mean_val_loss}")

    return losses, val_losses

def get_train_val_dataloaders(N_train=500, N_val=200, is_naive=False):
    # Generate train data
    x, y = generate_data(N_train, lower=-0.25, upper=1, std=0.02)

    # Generate validation data
    x_val, y_val = generate_data(N_val, lower=-0.25, upper=1, std=0.02)

    # make dataset and get dataloaders
    traindata = ToyDataset(x, y)
    valdata = ToyDataset(x_val, y_val)
    
    if is_naive:
        trainloader = DataLoader(traindata, batch_size=60, shuffle=True, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False)
        valloader = DataLoader(valdata, batch_size=60, shuffle=False, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False)
    else:
        trainloader = DataLoader(traindata, batch_size=60*M, shuffle=True, collate_fn=lambda x: train_collate_fn(x, M), drop_last=True)
        valloader = DataLoader(valdata, batch_size=60, shuffle=False, collate_fn=lambda x: test_collate_fn(x, M), drop_last=False)  
    
    return trainloader, valloader

def plot_loss(losses, val_losses):

    fig, ax = plt.subplots(1,2, figsize=(12,6))

    ax[0].plot(losses, label='Train loss')
    ax[0].set_title('Train loss')
    ax[0].set_xlabel('Iterations')
    ax[0].set_ylabel('Loss')
    ax[0].grid()

    ax[1].plot(val_losses, label='Validation loss', color='orange')
    ax[1].set_title('Validation loss')
    ax[1].set_xlabel('Iterations')
    ax[1].set_ylabel('Loss')
    ax[1].grid()

    plt.show()

We train one MIMO model and one Naive model with $M=[2,3,,4,5]$ subnetworks. 

In [ ]:
# train MIMO model with M = [2,3,4, 5]

Ms = [2, 3, 4, 5]

for M in Ms:
    MIMO_model = MIMONetwork(n_subnetworks=M)
    MIMO_model.apply(init_weights)
    
    optimizer = torch.optim.Adam(MIMO_model.parameters(), lr=3e-4)

    trainloader = DataLoader(traindata, batch_size=60*M, shuffle=True, collate_fn=lambda x: train_collate_fn(x, M), drop_last=True)
    valloader = DataLoader(valdata, batch_size=60, shuffle=False, collate_fn=lambda x: test_collate_fn(x, M), drop_last=False)

    losses, val_losses = train(MIMO_model, optimizer, epochs=5000, trainloader=trainloader, valloader=valloader, val_every_n_epochs=2, model_name=f'mimo_ensembles/MIMO_{M}_members')
    # plot loss 
    plot_loss(losses, val_losses)

In [ ]:
# train Naive model with M = [2,3,4]
Ms = [2, 3, 4, 5]

for M in Ms:
    naive_model = NaiveNetwork(n_subnetworks=M)
    naive_model.apply(init_weights)
    optimizer = torch.optim.Adam(naive_model.parameters(), lr=3e-4)

    naivetrainloader = DataLoader(traindata, batch_size=60, shuffle=True, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False)
    naivevalloader = DataLoader(valdata, batch_size=60, shuffle=False, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False)

    losses, val_losses = train(naive_model, optimizer, epochs=5000, trainloader=naivetrainloader, valloader=naivevalloader, val_every_n_epochs=2, model_name=f'naive_ensembles/Naive_{M}_members')
    # plot loss 
    plot_loss(losses, val_losses)


# Inference
We test the model on the test set and try to predict the function. We predict the mean outputs (of ensemble member) and of individual members.

In [ ]:
def inference(model, testloader):
    predictions = []
    pred_individual = []

    for test_x, test_y in testloader:
        output, individual_outputs = model(test_x.float())
        
        predictions.extend(list(output.detach().numpy()))
        pred_individual.extend(list(individual_outputs.detach().numpy()))

    return np.array(predictions), np.array(pred_individual)

# get predictions and individual predictions for MIMO and Naive models
def get_mimo_predictions(model_path, Ms, testdata, N_test=200):

    predictions_matrix = np.zeros((len(model_path), N_test))
    pred_individual_list = []

    for i, model in enumerate(model_path):

        M = Ms[i]
        testloader = DataLoader(testdata, batch_size=N_test, shuffle=False, collate_fn=lambda x: test_collate_fn(x, M), drop_last=False)

        model = torch.load(model)
        predictions, pred_individual = inference(model, testloader)

        predictions_matrix[i, :] = predictions
        pred_individual_list.append(pred_individual)
            
    return predictions_matrix, pred_individual_list

def get_naive_predictions(model_path, Ms, testdata, N_test=200):

    predictions_matrix = np.zeros((len(model_path), N_test))
    pred_individual_list = []

    for i, model in enumerate(model_path):

        M = Ms[i]
        testloader = DataLoader(testdata, batch_size=N_test, shuffle=False, collate_fn=lambda x: naive_collate_fn(x, M), drop_last=False)

        model = torch.load(model)
        predictions, pred_individual = inference(model, testloader)

        predictions_matrix[i, :] = predictions
        pred_individual_list.append(pred_individual)
            
    return predictions_matrix, pred_individual_list

Ms = [2, 3, 4]

mimo_path = [os.path.join("mimo_ensembles/", model) for model in ['MIMO_2_members.pt', 'MIMO_3_members.pt', 'MIMO_4_members.pt']]
mimo_pred_matrix, mimo_individual_list = get_mimo_predictions(mimo_path, Ms, testdata, N_test)
mimo_stds = np.array([np.std(pred, axis=1, ddof=1) for pred in mimo_individual_list])

naive_path = [os.path.join("naive_ensembles/", model) for model in ['Naive_2_members.pt', 'Naive_3_members.pt', 'Naive_4_members.pt']]
naive_pred_matrix, naive_individual_list = get_naive_predictions(naive_path, Ms, testdata, N_test)
naive_stds = np.array([np.std(pred, axis=1, ddof=1) for pred in naive_individual_list])



**Analysis**

We need to analyse the trained network to see if the MIMO network works as desired. We plot the mean predictions of each model and the 95% confidence intervals using the standard deviation computed 

In [ ]:
# plot data
fig, ax = plt.subplots(1,2, figsize=(18,12))

### plot mimo ###
ax[0].plot(x, y, '.', label='Train data', color='orange')
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[0].grid()
ax[0].plot(x_test, line, '--', label='true function', color='red')
# plot test data
ax[0].plot(x_test, y_test, '.', label='Test data', color='black')

# plot predicitons with confidence intervals
for i in range(len(Ms)):
    ax[0].plot(x_test, mimo_pred_matrix[i], '-', label=f'Mean MIMO Predictions with {Ms[i]} members', linewidth=2)
    ax[0].fill_between(x_test, mimo_pred_matrix[i] - 1.96*mimo_stds[i], mimo_pred_matrix[i] + 1.96*mimo_stds[i], alpha=0.2, label=f'Confidence Interval with {Ms[i]} members')

ax[0].legend()

### naive ###
ax[1].plot(x, y, '.', label='Train data', color='orange')
ax[1].set_xlabel('x')
ax[1].set_ylabel('y')
ax[1].grid()
ax[1].plot(x_test, line, '--', label='true function', color='red')
# plot test data
ax[1].plot(x_test, y_test, '.', label='Test data', color='black')

# # plot predicitons with confidence intervals
for i in range(len(Ms)):
    ax[1].plot(x_test, naive_pred_matrix[i], '-', label=f'Mean naive Predictions with {Ms[i]} members', linewidth=2)
    ax[1].fill_between(x_test, naive_pred_matrix[i] - 1.96*naive_stds[i], naive_pred_matrix[i] + 1.96*naive_stds[i], alpha=0.2, label=f'Confidence Interval with {Ms[i]} members')
ax[1].legend()


plt.show()


The plot shows the mean predictions for the MIMO and naive models with $M=[2,3,4]$ ensemble members, illustrated with a 95% confidence interval. 
The confidence interval is constructed using the variance of individual predictions for each subnetwork in a model:


# Classification
We now move on to the classification task. To keep close to the MIMO paper we use CIFAR-10 for data:

We try to follow the SmallCNN example from Fort et al. that is also used in the MIMO paper. Some parts are not well described in the paper, such as the exact structure of the input and output layers. So we have simply made some decision that we think are appropriate in those regards.

In [ ]:
#load/download CIFAR10
# https://github.com/kuangliu/pytorch-cifar/issues/19 normalisation values
transform = transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

CIFAR_traindata = torchvision.datasets.CIFAR10(root="../data/", train=True, transform = transform, download=False)
CIFAR_train, CIFAR_val = torch.utils.data.random_split(CIFAR_traindata, [int(len(CIFAR_traindata)*0.9), int(len(CIFAR_traindata)*0.1)])

CIFAR_test = torchvision.datasets.CIFAR10(root="../data/", train=False, transform = transform, download=False)


Images from the CIFAR10 dataset are 32x32 colour images. Below, some examples are visualised with their label. 

In [ ]:
inv_transform = transforms.Normalize(mean=[-0.4914/0.247, -0.4822/0.243, -0.4465/0.261], std=[1/0.247, 1/0.243, 1/0.261])
label_dict = {0: "airplane", 
              1: "automobile",
              2: "bird",
              3: "cat",
              4: "deer",
              5: "dog",
              6: "frog",
              7: "horse",
              8: "ship",
              9: "truck"}

# plot some images
fig, ax = plt.subplots(1,5, figsize=(18,12))
for i in range(5):
    x, y = CIFAR_train[i+25]
    img = inv_transform(x) # unnormalise
    ax[i].imshow(img.permute(1,2,0))
    ax[i].set_title(f'Label: {label_dict[y]}')
    ax[i].set_xticks([])
    ax[i].set_yticks([])


In the [Github repository](https://github.com/google/edward2/blob/7a11c9782b1e72eef8e2f4fe7914827b799b692f/experimental/mimo/imagenet_model.py#L132-L138) they use the following operations to manipulate the input shape of ImageNet images:

In [ ]:
M = 3
input_shape = (M, 32, 32, 3)
input_shape = list(input_shape)
inp = torch.ones(input_shape)
x = inp.permute(1,2,3,0)
x = torch.ones(list(input_shape[1:-1]) + [input_shape[-1] * M])

print(x.shape)

In order to apply this to the CIFAR10 dataset, we extrapolate that we need to concatenate the ``ensemble_size`` dimension and channel dimension. This is what we will do in the collate functions below.

The collate function needs to be different for the CIFAR data. Pytorch has built-in CIFAR dataset so no dataset class is needed. The collate function is responsible for drawing M samples at a time so that each of the subnetworks gets a different input during training and the same input during inference.

During training, the input image $x$ naturally has shape $(batch\_size \cdot M, C, H, W)$ and the input label has shape $(batch\_size \cdot M). With the collate fucntion during training, the input image and label is split into $M$ chunks and concatenated in the channel dimension (dim = 1). 

With the collate function during testing, the input image and label is duplicated $M$ times in dim 1. 

In [ ]:
#Collate functions
def C_train_collate_fn(batch, M):
    """Collate function for training MIMO on CIFAR classification"""
    
    x, y = zip(*batch)

    x, y = torch.stack(list(x)), torch.tensor(y)
    x = torch.cat(torch.chunk(x, M, dim=0), dim=1)
    y = torch.stack(torch.chunk(y, M, dim=0), dim=1)

    return x, y

def C_test_collate_fn(batch, M):
    """Collate function for testing MIMO on CIFAR classification"""
    
    x, y = zip(*batch)
    x, y = torch.stack(list(x)), torch.tensor(y)
    x = x.repeat(1, M, 1, 1)
    y = y[:,None].repeat(1,M)
    
    return x, y

def C_Naive_train_collate_fn(batch, M):
    """Collate function for training Naive multiheaded on CIFAR classification"""

    x, y = zip(*batch)

    x, y = torch.stack(list(x)), torch.tensor(y)
    y = y[:,None].repeat(1,M)

    return x, y

def C_Naive_test_collate_fn(batch, M):
    """Collate function for testing Naive multiheaded on CIFAR classsification"""

    x, y = zip(*batch)
    x, y = torch.stack(list(x)), torch.tensor(y)
    y = y[:,None].repeat(1,M)

    return x, y

The network is constructed in the following way:
- The input is put through $3$ convolutions with channels going from $3 \rightarrow 16$, $16 \rightarrow 32$, $32 \rightarrow 32$
- the output from the convolutions should be shape $( batch\_size, 32, 32, 32)$, because we use ``padding=1``. This is then flattened before being inputted into the linear layer
- after convolution there are $2$ linear layers where the dimensions is reduced down to $M\cdot 10$ - 10 for each class, for each subnetwork'

In the forward pass, the output are LogSoftmax probabilities. 

A separate ``ìnference`` function is created to output the actual predictions of 1) individual subnetworks and 2) the ensemble as a whole.

In [ ]:
# Initialise Network
class C_MIMONetwork(nn.Module):
    def __init__(self, n_subnetworks):
        super().__init__()
        self.n_subnetworks = n_subnetworks
        self.in_channels = 3
        self.channels1 = 16
        self.channels2 = 32

        self.conv = torch.nn.Sequential(
            nn.Conv2d(self.in_channels*self.n_subnetworks, self.channels1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(self.channels1, self.channels2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(self.channels2, self.channels2, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.output = torch.nn.Sequential(
            nn.Linear(self.channels2 * 32 * 32, 128), # dim: self.channels2 x width x height
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(128, self.n_subnetworks*10)
        )

    def forward(self, x):

        x = self.conv(x)
        # reshape to fit into linear layer
        x = x.reshape(x.size(0), -1)
        x = self.output(x)

        # reshape to batch_size x M x 10
        x = x.reshape(x.size(0), self.n_subnetworks, -1)
        # Log-softmax over the last dimension (because we are using NLL loss)
        x = nn.LogSoftmax(dim=2)(x)

        # get individual outputs 
        # during training, we want each subnetwork to to clasify their corresponding inputs
        individual_outputs = torch.argmax(x, dim=2) # dim : batch_size x M
        
        # get ensemble output
        # during inference, we mean the softmax probabilities over all M subnetworks and then take the argmax
        output = torch.mean(x, dim=1).argmax(dim=1) # dim : batch_size
        
        x = x.permute(1,0,2)

        return x, output, individual_outputs

    # def inference(self, x):

    #     x = self.conv(x)
    #     # reshape to fit into linear layer
    #     x = x.reshape(x.size(0), -1)
    #     x = self.output(x)

    #     # reshape to batch_size x M x 10
    #     x = x.reshape(x.size(0), M, -1)
    #     # Log-softmax over the last dimension (because we are using NLL loss)
    #     x = nn.LogSoftmax(dim=2)(x)

    #     # get individual outputs 
    #     # during training, we want each subnetwork to to clasify their corresponding inputs
    #     individual_outputs = torch.argmax(x, dim=2) # dim : batch_size x M
        
    #     # get ensemble output
    #     # during inference, we mean the softmax probabilities over all M subnetworks and then take the argmax
    #     output = torch.mean(x, dim=1).argmax(dim=1) # dim : batch_size

    #     return output, individual_outputs

In [ ]:
# Initialise Network
class C_NaiveNetwork(nn.Module):
    def __init__(self, n_subnetworks):
        super().__init__()
        self.n_subnetworks = n_subnetworks
        self.in_channels = 3
        self.channels1 = 16
        self.channels2 = 32

        self.conv = torch.nn.Sequential(
            nn.Conv2d(self.in_channels, self.channels1, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(self.channels1, self.channels2, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(self.channels2, self.channels2, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.output = torch.nn.Sequential(
            nn.Linear(self.channels2 * 32 * 32, 128), # dim: self.channels2 x width x height
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(128, self.n_subnetworks*10)
        )

    def forward(self, x):

        x = self.conv(x)
        # reshape to fit into linear layer
        x = x.reshape(x.size(0), -1)
        x = self.output(x)

        # reshape to batch_size x M x 10
        x = x.reshape(x.size(0), self.n_subnetworks, -1)
        # Log-softmax over the last dimension (because we are using NLL loss)
        x = nn.LogSoftmax(dim=2)(x)
        
        # get individual outputs 
        # during training, we want each subnetwork to to clasify their corresponding inputs
        individual_outputs = torch.argmax(x, dim=2) # dim : batch_size x M
        
        # get ensemble output
        # during inference, we mean the softmax probabilities over all M subnetworks and then take the argmax
        output = torch.mean(x, dim=1).argmax(dim=1) # dim : batch_size

        x = x.permute(1,0,2)

        return x, output, individual_outputs


In [ ]:
# seed workers for reproducibility
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

# train on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
def train_classification(model, optimizer, trainloader, valloader, epochs=500, model_name='MIMO', val_every_n_epochs=10, checkpoint_every_n_epochs=20, loss_fn = nn.NLLLoss(reduction='mean'), device='cpu'):
    losses = []
    val_losses = []
    val_checkpoint_list = []

    best_val_loss = np.inf

    for e in tqdm(range(epochs)):
        
        for x_, y_ in trainloader:

            x_,y_ = x_.float().to(device), y_.long().to(device)

            model.train()

            optimizer.zero_grad()

            log_prob, _, _ = model(x_)
            
            # sum loss per subnetwork
            # mean is already taken over the batch, because we use reduction = 'mean' in the loss function
            loss = 0
            for log_p, y in zip(log_prob, y_.T):
                # print(log_p.shape)
                # print(y.shape)
                loss += loss_fn(log_p, y)

            loss.backward()
            optimizer.step()

            losses.append(loss.item())  

        if (e+1) % val_every_n_epochs == 0:
            model.eval()

            val_loss_list = []
            
            with torch.no_grad():
                for val_x, val_y in valloader:
                    val_x, val_y = val_x.float().to(device), val_y.long().to(device)
                    log_prob, _, _ = model(val_x)
                
                    val_loss = 0
                    for log_p, y in zip(log_prob, val_y.T):
                        val_loss += loss_fn(log_p, y)

                    val_loss_list.append(val_loss.item())
                if (e+1) % checkpoint_every_n_epochs == 0:
                    val_checkpoint_list.append(log_prob[0,:,:])

            val_losses.extend(val_loss_list)
            mean_val_loss = np.mean(val_loss_list)
            if mean_val_loss < best_val_loss:
                best_val_loss = mean_val_loss
                torch.save(model, f'{model_name}_{model.n_subnetworks}.pt')
            # print(f"Mean validation loss at epoch {e}: {mean_val_loss}")
    torch.save(torch.stack(val_checkpoint_list), f'{model_name}_{model.n_subnetworks}_checkpoints.pt')

    return losses, val_losses, val_checkpoint_list


In [ ]:
# train classification MIMO model with M = [2,3,4]

loss_fn = nn.NLLLoss(reduction='mean')
Ms = [2, 3, 4]
batch_size = 512

for M in Ms:
    C_MIMO_model = C_MIMONetwork(n_subnetworks=M)
    C_MIMO_model.apply(init_weights)
    C_MIMO_model = C_MIMO_model.to(device)
    
    optimizer = torch.optim.Adam(C_MIMO_model.parameters(), lr=3e-4)
    
    C_trainloader = DataLoader(CIFAR_train, batch_size=batch_size*M, shuffle=True, collate_fn=lambda x: C_train_collate_fn(x, M), drop_last=True, worker_init_fn=seed_worker, generator=g)
    C_valloader = DataLoader(CIFAR_val, batch_size=batch_size, shuffle=False, collate_fn=lambda x: C_test_collate_fn(x, M), drop_last=False)

    losses, val_losses, val_checkpoint_list = train_classification(C_MIMO_model, optimizer, C_trainloader, C_valloader, epochs=30, model_name='C_MIMO', val_every_n_epochs=2, checkpoint_every_n_epochs=20, loss_fn = loss_fn, device=device)
    # plot loss 
    plot_loss(losses, val_losses)

In [ ]:
# train classification Naive model with M = [2,3,4]

loss_fn = nn.NLLLoss(reduction='mean')
Ms = [2, 3, 4]
batch_size = 512

for M in Ms:
    C_Naive_model = C_NaiveNetwork(n_subnetworks=M)
    C_Naive_model.apply(init_weights)
    C_Naive_model = C_Naive_model.to(device)
    
    Naive_optimizer = torch.optim.SGD(C_Naive_model.parameters(), lr=1e-3, weight_decay=3e-4)
    
    C_Naive_trainloader = DataLoader(CIFAR_train, batch_size=batch_size, shuffle=True, collate_fn=lambda x: C_Naive_train_collate_fn(x, M), drop_last=True, worker_init_fn=seed_worker, generator=g)
    C_Naive_valloader = DataLoader(CIFAR_val, batch_size=batch_size, shuffle=False, collate_fn=lambda x: C_Naive_test_collate_fn(x, M), drop_last=False)

    naive_losses, naive_val_losses, val_checkpoint_list = train_classification(C_Naive_model, Naive_optimizer, C_Naive_trainloader, C_Naive_valloader, epochs=30, model_name="C_Naive", val_every_n_epochs=2, checkpoint_every_n_epochs=20, loss_fn = loss_fn, device=device)
    # plot loss 
    plot_loss(losses, val_losses)

# Inference

We will predict on the test images.

In [ ]:
batch_size = 512
C_testloader = DataLoader(CIFAR_test, batch_size=batch_size, shuffle=False, collate_fn=lambda x: C_test_collate_fn(x, M), drop_last=False)
C_Naive_testloader = DataLoader(CIFAR_test, batch_size=batch_size, shuffle=False, collate_fn=lambda x: C_Naive_test_collate_fn(x, M), drop_last=False)

In [ ]:
# weight distribution

weights_mimo = []
for param in C_MIMO_model.parameters():
    weights_mimo.extend(param.flatten().cpu().detach().numpy())

weights_naive = []
for param in C_Naive_model.parameters():
    weights_naive.extend(param.flatten().cpu().detach().numpy())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.hist(weights_mimo, bins=50, alpha=0.5, label='MIMO_model')
ax1.set_xlabel('Weight Value')
ax1.set_ylabel('Frequency')
ax1.set_title('Weight Distribution - Classification MIMO Model')

ax2.hist(weights_naive, bins=50, alpha=0.5, label='naive_model')
ax2.set_xlabel('Weight Value')
ax2.set_ylabel('Frequency')
ax2.set_title('Weight Distribution - Classification Naive Model')

plt.tight_layout()
plt.show()

In [ ]:
C_mimo_path = [os.path.join("C_mimo_ensembles/", model) for model in ['C_MIMO_2.pt', 'C_MIMO_3.pt', 'C_MIMO_4.pt']]

for i, model in enumerate(C_mimo_path):
    M = i+2
    C_MIMO_model = torch.load(model)
    C_MIMO_model = C_MIMO_model.to(device)

    C_testloader = DataLoader(CIFAR_test, batch_size=batch_size, shuffle=False, collate_fn=lambda x: C_test_collate_fn(x, M), drop_last=False)

    predictions = []
    ground_truth = []
    confidence = []

    for x_test, y_test in C_testloader:
        x_test, y_test = x_test.to(device), y_test.to(device)
        _, output, individual_outputs = C_MIMO_model.forward(x_test)
        
        predictions.extend(list(output.cpu().detach().numpy()))
        ground_truth.extend(list(y_test[:,0].cpu().detach().numpy()))

    accuracy = np.mean(np.array(predictions) == np.array(ground_truth))
    print(f"Accuracy for MIMO with {M} members:", accuracy)

In [ ]:
C_naive_path = [os.path.join("C_naive_ensembles/", model) for model in ['C_Naive_2.pt', 'C_Naive_3.pt', 'C_Naive_4.pt']]

for i, model in enumerate(C_naive_path):
    M = i+2
    C_Naive_model = torch.load(model)
    C_Naive_model = C_Naive_model.to(device)

    C_Naive_testloader = DataLoader(CIFAR_test, batch_size=batch_size, shuffle=False, collate_fn=lambda x: C_Naive_test_collate_fn(x, M), drop_last=False)

    naive_predictions = []
    naive_ground_truth = []
    naive_confidence = []

    for x_test, y_test in C_Naive_testloader:
        x_test, y_test = x_test.to(device), y_test.to(device)
        _, output, individual_outputs = C_Naive_model.forward(x_test)
        
        naive_predictions.extend(list(output.cpu().detach().numpy()))
        naive_ground_truth.extend(list(y_test[:,0].cpu().detach().numpy()))

    accuracy = np.mean(np.array(naive_predictions) == np.array(naive_ground_truth))
    print(f"Accuracy for naive model with {M} members:", accuracy)

***Plots for Image Classifiers***

We want to illustrate the overconfidence of the Image classifier models. To do this we make reliability diagrams as described in Gou et al. 

The reliability diagram has accuracy on the y-axis and confidence on the x-axis. We sample some test points, then bin them based on the models confidence in its prediction for those points. Thus we will have accuracy sorted in bins like so:
$$acc(B_m)=\frac{1}{|B_m|}\sum_{i\in B_m} \mathbf{1}(\hat{y}_i=y_i)$$
where $\mathbf{1}(\hat{y}_i=y_i)$ is an indicator function for a correct prediction and $B_m$ is the m-th bin of $M$ bins with width $\frac{1}{M}$.

The confidence is likewise calculated as 
$$conf(B_m)=\frac{1}{|B_m|}\sum_{i\in B_m} \hat{p}_i$$
where $\hat{p_i}$ is the confidence for sample $i$. In this case, since we have no better measure, the softmax probabilities will be used as confidence scores.

A perfectly calibrated model will have $acc(B_m) = conf(B_m)$ but since neural networks are typically overconfident we will expect to see $acc(B_m) < conf(B_m)$


In [ ]:
#Code for generating reliability diagram:
bins_range = np.arange(0, 1, 0.1)
conf_step_height = np.zeros(10)
acc_step_height = np.zeros(10)
for i in range(len(bins_range)-1):
    loc = np.where(confidence>=bins_range[i] and confidence<bins_range[i+1])
    conf_step_height[i] = np.mean(confidence[loc])
    acc_step_height[i] = np.mean(accuracy[loc])




naive_conf_step_height = np.zeros(10)
naive_acc_step_height = np.zeros(10)
for i in range(len(bins_range)-1):
    loc = np.where(naive_confidence>=bins_range[i] and naive_confidence<bins_range[i+1])
    naive_conf_step_height[i] = np.mean(confidence[loc])
    naive_acc_step_height[i] = np.mean(accuracy[loc])



ax, fig = plt.subplots(2,1, sharey=True)
ax.set_xlabel("Confidence")
ax.set_ylabel("Accuracy")
ax[0].stairs(conf_step_height, bins_range, hatch="//")
ax[0].stairs(acc_step_height, bins_range, fill = True)
ax[0].set_title("Reliability Plot MIMO")

ax[1].stairs(conf_step_height, bins_range, hatch="//", legend='Gap')
ax[1].stairs(acc_step_height, bins_range, fill = True, legend="Outputs")
ax[1].set_title("Reliability Plot Naive Multiheaded")
plt.show()

In order to observe the diversity of the subnetworks in the MIMO model, we use function space plots like in Fort et al.
This lets us visualize how each subnetwork moves in function space.

The plots are made with the following steps:
- At certain model checkpoints, get softmax predictions and use tSNE to reduce the softmaxoutput to 2d so that you can plot it.
- first separate checkpoint output so that each subnetwork is isolated
- then apply the same t-SNE algorithm to all of them
- then plot them together

tips:
- TNSE works best with a reasonable dimensionality (around 50 dimensions) [citation needed] so we limit the number of predictions. We have a large batch size and 10 class prediction pr data-point. Consider this when selecting how many predictions to keep.


In [ ]:
print(val_checkpoint_list.shape)


In [ ]:
val_checkpoint_list = torch.stack(val_checkpoint_list[:,:5,:]).flatten()
tSNE = TSNE(val_checkpoint_list.shape)
val_checkpoint_list2d = tSNE.fit_transform(val_checkpoint_list)
plt.plot(val_checkpoint_list)



In [ ]:
torch.stack(val_checkpoint_list).shape

In [ ]:
512*392

In [ ]:
val_checkpoint_list[0].shape